In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sys

# Para rodar no notebook
%matplotlib inline

# Adiciona o diretório src ao sys.path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# Importar o TensorFlow Addons
import tensorflow_addons as tfa

# Ignorar avisos desnecessários
import warnings
warnings.filterwarnings('ignore')


# Importar as funções de data augmentation
from funcoes_augmentation import load_and_preprocess_image, random_rotation, adjust_brightness, flip_horizontal, random_zoom, random_shear

c:\Users\arthu\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# Treinamento do Modelo de Classificação de Artefatos Arqueológicos

Neste notebook, realizaremos o treinamento de um modelo de classificação de imagens de artefatos arqueológicos utilizando uma rede neural convolucional baseada na EfficientNet. O objetivo é avaliar o impacto de diferentes técnicas de data augmentation no desempenho do modelo, testando cada técnica individualmente e buscando os níveis ótimos antes de combiná-las.

---

**Objetivos:**

- Treinar um modelo baseline sem data augmentation.
- Aplicar técnicas de data augmentation individualmente:
  - Rotação
  - Alteração de Brilho
  - Espelhamento Horizontal
  - Zoom In e Zoom Out
  - Cisalhamento (Shear)
- Comparar o desempenho do modelo com cada técnica.
- Identificar as melhores técnicas e combiná-las.
- Documentar os resultados para análise posterior.

---

In [2]:
# Diretórios de dados
train_dir = '../dados/brutos/train'
test_dir = '../dados/brutos/test'

# Lista das classes
classes = ['classe1_Vidro', 'classe2_Ceramica', 'classe3_Litico', 'classe4_Louca']

# Mapeamento das classes para índices
class_indices = {cls_name: idx for idx, cls_name in enumerate(classes)}

# Parâmetros do modelo
batch_size = 32
img_height = 256
img_width = 256
num_classes = len(classes)
epochs = 10  

In [3]:
def get_image_paths_and_labels(data_dir, classes):
    image_paths = []
    labels = []
    for cls in classes:
        cls_dir = os.path.join(data_dir, cls)
        cls_index = class_indices[cls]
        for img_name in os.listdir(cls_dir):
            img_path = os.path.join(cls_dir, img_name)
            image_paths.append(img_path)
            labels.append(cls_index)
    return image_paths, labels

# Dados de treinamento
train_image_paths, train_labels = get_image_paths_and_labels(train_dir, classes)

# Dados de validação (teste)
test_image_paths, test_labels = get_image_paths_and_labels(test_dir, classes)

# Converter para tensores
train_image_paths = tf.constant(train_image_paths)
train_labels = tf.constant(train_labels)

test_image_paths = tf.constant(test_image_paths)
test_labels = tf.constant(test_labels)

Criamos o dataset de teste (validação) sem data augmentation.

In [4]:
# Dataset de validação
test_ds = tf.data.Dataset.from_tensor_slices((test_image_paths, test_labels))

test_ds = test_ds.map(
    lambda x, y: (load_and_preprocess_image(x), y)
).batch(batch_size).prefetch(tf.data.AUTOTUNE)


# Definição do Modelo

Carregar o Modelo Base EfficientNet

In [5]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models

def create_model(num_classes):
    # Carregar o modelo base
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    base_model.trainable = False  # Transfer learning - congelar as camadas convolucionais

    # Construir a arquitetura do modelo
    inputs = layers.Input(shape=(img_height, img_width, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    return model

# Compilação do Modelo

Definimos o otimizador, a função de perda e as métricas.

In [6]:
# Criar o modelo
model = create_model(num_classes)

# Compilar o modelo
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

16705208/16705208 [==============================] - 2s 0us/step


# Treinamento do Modelo

Vamos treinar o modelo em diferentes cenários:

- Baseline: Sem data augmentation.
- Com cada técnica de data augmentation individualmente.

Criar o Dataset de Treinamento Sem Data Augmentation:

In [7]:
# Dataset de treinamento sem data augmentation
train_ds_baseline = tf.data.Dataset.from_tensor_slices((train_image_paths, train_labels))

train_ds_baseline = train_ds_baseline.map(
    lambda x, y: (load_and_preprocess_image(x), y)
).shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

Treinar o Modelo

In [ ]:
# Recriar o modelo para garantir que começamos do zero
model = create_model(num_classes)

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Treinar o modelo
history_baseline = model.fit(
    train_ds_baseline,
    validation_data=test_ds,
    epochs=epochs
)

# Salvar o histórico
histories = {}
histories['baseline'] = history_baseline

Epoch 1/10
 3/48 [>.............................] - ETA: 1:10 - loss: 1.2751 - accuracy: 0.3542